Install the libraries

In [1]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00


In [2]:
from collections import defaultdict
import gdown
import gzip
import json
import random
import pickle

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
sns.set(style='ticks', font_scale=1.2)

In [3]:
#import the libraries for importing and dealing with data
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#import libraries for processing
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [6]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Load German Dataset and Clean it**

In [7]:
#load and read the training TEXT From file
germantrainingSet = pd.read_csv('drive/MyDrive/germanOriginal.txt', on_bad_lines='skip', sep=';', header=None)

In [8]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
germantrainingSet=germantrainingSet.rename(columns={0: 'Tweet'})

In [9]:
germantrainingSet.head()

,Tweet
0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,@ahrens_theo fröhlicher gruß aus der schönsten...
3,@dushanwegner Amis hätten alles und jeden gewä...
4,@spdde kein verläßlicher Verhandlungspartner. ...


In [10]:
def check_for_abuse(row):
    if row.endswith("ABUSE"):
        return 1
    else:
        return 0

In [11]:
germantrainingSet['Label'] = germantrainingSet['Tweet'].apply(check_for_abuse)

In [12]:
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].apply(lambda x: ' '.join(x.split()[:-1]))

In [13]:
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].str.replace('OTHER', '')
germantrainingSet['Tweet'] = germantrainingSet['Tweet'].str.replace('OFFENSE', '')

In [14]:
germantrainingSet.head()

,Tweet,Label
0,"@corinnamilborn Liebe Corinna, wir würden dich...",0
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,0
2,@ahrens_theo fröhlicher gruß aus der schönsten...,0
3,@dushanwegner Amis hätten alles und jeden gewä...,0
4,@spdde kein verläßlicher Verhandlungspartner. ...,0


In [15]:
germantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4811 entries, 0 to 4810
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   4811 non-null   object
 1   Label   4811 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 75.3+ KB


In [16]:
count = (germantrainingSet['Label'] == 1).sum()

# The condition (df['Label'] == 1) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 1 in 'Label' column:", count)

Number of rows with value 1 in 'Label' column: 995


In [17]:
countZero = (germantrainingSet['Label'] == 0).sum()

# The condition (df['Label'] == 0) checks for rows with value 1, and sum() counts the True values

print("Number of rows with value 0 in 'Label' column:", countZero)

Number of rows with value 0 in 'Label' column: 3816


In [18]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

germantrainingSet["tweet_without_url"] = germantrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [19]:
germantrainingSet.head()

,Tweet,Label,tweet_without_url
0,"@corinnamilborn Liebe Corinna, wir würden dich...",0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,@Martin28a Sie haben ja auch Recht. Unser Twee...,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,@ahrens_theo fröhlicher gruß aus der schönsten...,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,@dushanwegner Amis hätten alles und jeden gewä...,0,@dushanwegner Amis hätten alles und jeden gewä...
4,@spdde kein verläßlicher Verhandlungspartner. ...,0,@spdde kein verläßlicher Verhandlungspartner. ...


**Load Spanish Dataset and Clean it**

In [20]:
#load and read the training TEXT From file
spanishtrainingSet = pd.read_csv('drive/MyDrive/spanishOriginal.txt', on_bad_lines='skip', sep=';', header=None)

In [21]:
#drop the columns not needed
spanishtrainingSet.drop(columns=0, inplace=True)
spanishtrainingSet.drop(columns=1, inplace=True)
spanishtrainingSet.drop(columns=3,inplace=True)

In [22]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
spanishtrainingSet=spanishtrainingSet.rename(columns={2: 'Tweet'})
spanishtrainingSet=spanishtrainingSet.rename(columns={4: 'Label'})

In [23]:
spanishtrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5851 entries, 0 to 5850
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   5851 non-null   object
 1   Label   5851 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 91.5+ KB


**Remove all links**

In [24]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

spanishtrainingSet["tweet_without_url"] = spanishtrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [25]:
spanishtrainingSet.head()

,Tweet,Label,tweet_without_url
0,Ismael es egocentrico porque se vuelve loca si...,0,Ismael es egocentrico porque se vuelve loca si...
1,..ya tardaba en salir quien pronunciase nombre...,0,..ya tardaba en salir quien pronunciase nombre...
2,"(Esto no es un discurso político y razonado, o...",0,"(Esto no es un discurso político y razonado, o..."
3,"Muy despreciados,siiii,pero todos vestidos de ...",1,"Muy despreciados,siiii,pero todos vestidos de ..."
4,marica explicame porque a veces no te entiendo...,1,marica explicame porque a veces no te entiendo...


**Load Italian Dataset and Clean it **

In [26]:
#load and read the training TEXT From file
italiantrainingSet = pd.read_csv('drive/MyDrive/italianOriginal.tsv', on_bad_lines='skip', sep='\t')

In [27]:
#drop the columns not needed
italiantrainingSet.drop(columns='id', inplace=True)
italiantrainingSet.drop(columns='stereotype', inplace=True)

In [28]:
#rename the column text to 'Tweet' and column hs to 'Label
italiantrainingSet=italiantrainingSet.rename(columns={'text ': 'Tweet'})
italiantrainingSet=italiantrainingSet.rename(columns={'hs': 'Label'})

In [29]:
#function remove_urls cleans the Urls from the Tweets and put in a seperate column
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

italiantrainingSet["tweet_without_url"] = italiantrainingSet["Tweet"].apply(lambda text: remove_urls(text))

In [30]:
italiantrainingSet.head()

,Tweet,Label,tweet_without_url
0,"È terrorismo anche questo, per mettere in uno ...",0,"È terrorismo anche questo, per mettere in uno ..."
1,@user @user infatti finché ci hanno guadagnato...,0,@user @user infatti finché ci hanno guadagnato...
2,"Corriere: Tangenti, Mafia Capitale dimenticata...",0,"Corriere: Tangenti, Mafia Capitale dimenticata..."
3,"@user ad uno ad uno, perché quando i migranti ...",0,"@user ad uno ad uno, perché quando i migranti ..."
4,Il divertimento del giorno? Trovare i patrioti...,0,Il divertimento del giorno? Trovare i patrioti...


In [31]:
italiantrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6837 entries, 0 to 6836
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Tweet              6837 non-null   object
 1   Label              6837 non-null   int64 
 2   tweet_without_url  6837 non-null   object
dtypes: int64(1), object(2)
memory usage: 160.4+ KB


**Merge the Data**

In [32]:
germantrainingSet.shape

(4811, 3)

In [33]:
#drop the columns not needed
germantrainingSet.drop(columns='Tweet', inplace=True)
#germantrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#germantrainingSet.drop(columns='tweet_lem', inplace=True)

In [34]:
germantrainingSet.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,0,@dushanwegner Amis hätten alles und jeden gewä...
4,0,@spdde kein verläßlicher Verhandlungspartner. ...


In [35]:
#figure out the shape of spanish set
spanishtrainingSet.shape

(5851, 3)

In [36]:
#drop the columns not needed
spanishtrainingSet.drop(columns='Tweet', inplace=True)
#spanishtrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#spanishtrainingSet.drop(columns='tweet_lem', inplace=True)

In [37]:
spanishtrainingSet.head()

,Label,tweet_without_url
0,0,Ismael es egocentrico porque se vuelve loca si...
1,0,..ya tardaba en salir quien pronunciase nombre...
2,0,"(Esto no es un discurso político y razonado, o..."
3,1,"Muy despreciados,siiii,pero todos vestidos de ..."
4,1,marica explicame porque a veces no te entiendo...


In [38]:
#figure out the shape of italian set
italiantrainingSet.shape

(6837, 3)

In [39]:
#drop the columns not needed
italiantrainingSet.drop(columns='Tweet', inplace=True)
#italiantrainingSet.drop(columns='tweet_w/o_url', inplace=True)
#italiantrainingSet.drop(columns='tweet_lem', inplace=True)

In [40]:
#rename the column 2 to 'Tweet' and column 4 to 'Label
italiantrainingSet=italiantrainingSet.rename(columns={1: 'tweet_without_url'})


In [41]:
italiantrainingSet.head()

,Label,tweet_without_url
0,0,"È terrorismo anche questo, per mettere in uno ..."
1,0,@user @user infatti finché ci hanno guadagnato...
2,0,"Corriere: Tangenti, Mafia Capitale dimenticata..."
3,0,"@user ad uno ad uno, perché quando i migranti ..."
4,0,Il divertimento del giorno? Trovare i patrioti...


In [42]:
germanspanish_merged = pd.merge(germantrainingSet, spanishtrainingSet, how="outer", on=["Label", "tweet_without_url"])

In [43]:
germanspanish_merged.shape

(10662, 2)

In [44]:
germanspanish_merged.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,0,@dushanwegner Amis hätten alles und jeden gewä...
4,0,@spdde kein verläßlicher Verhandlungspartner. ...


In [45]:
print(germanspanish_merged)

       Label                                  tweet_without_url
0          0  @corinnamilborn Liebe Corinna, wir würden dich...
1          0  @Martin28a Sie haben ja auch Recht. Unser Twee...
2          0  @ahrens_theo fröhlicher gruß aus der schönsten...
3          0  @dushanwegner Amis hätten alles und jeden gewä...
4          0  @spdde kein verläßlicher Verhandlungspartner. ...
...      ...                                                ...
10657      0  Sea independentista o constitucionalista, la d...
10658      0  @Bernithedude Llave de gobierno de qué si hay ...
10659      0  @13Pardis @guenhwyvarblack Los carlinos son un...
10660      0  Aplausos en el partido de Colau cuando anuncia...
10661      0  @Rafa_Hernando La Constitución es independenti...

[10662 rows x 2 columns]


In [46]:
germanspanishitalian_merged = pd.merge(germanspanish_merged, italiantrainingSet, how="outer", on=["Label", "tweet_without_url"])

In [47]:
germanspanishitalian_merged.shape

(17499, 2)

In [48]:
print(germanspanishitalian_merged)

       Label                                  tweet_without_url
0          0  @corinnamilborn Liebe Corinna, wir würden dich...
1          0  @Martin28a Sie haben ja auch Recht. Unser Twee...
2          0  @ahrens_theo fröhlicher gruß aus der schönsten...
3          0  @dushanwegner Amis hätten alles und jeden gewä...
4          0  @spdde kein verläßlicher Verhandlungspartner. ...
...      ...                                                ...
17494      0  Gli stati nazionali devono essere pronti a rin...
17495      0  Il ministro dell'interno della Germania #Horst...
17496      0  #Salvini: In Italia troppi si sono montati la ...
17497      0  @user @user Chi giubila in buona fede non ha c...
17498      0  I giovani cristiani in #Etiopia sono indotti d...

[17499 rows x 2 columns]


In [49]:
germanspanishitalian_merged.head()

,Label,tweet_without_url
0,0,"@corinnamilborn Liebe Corinna, wir würden dich..."
1,0,@Martin28a Sie haben ja auch Recht. Unser Twee...
2,0,@ahrens_theo fröhlicher gruß aus der schönsten...
3,0,@dushanwegner Amis hätten alles und jeden gewä...
4,0,@spdde kein verläßlicher Verhandlungspartner. ...


Rename Label to 'label'

In [50]:
germanspanishitalian_merged = germanspanishitalian_merged.rename(columns={'Label': 'label'})

In [51]:
#print(germanspanishitalian_merged["tweetwithoutstop"])

In [52]:
#print(germanspanishitalian_merged['label'])

In [53]:
X = germanspanishitalian_merged["tweet_without_url"].values.tolist()

In [54]:
y = germanspanishitalian_merged['label'].values.tolist()

**Split the text**

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 0)

In [56]:
train_texts = X_train
train_labels = y_train

test_texts = X_test
test_labels = y_test

In [57]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(13999, 13999, 3500, 3500)

In [58]:
train_labels[0], train_texts[0]

(1,
 '@user Possibile che si debba passare ai fatti per far capire a tutti che ci siamo stufati di essere presi per il culo e per il portafogli? Io non voglio dare un euro a migranti, clandestini, rom e feccia varia. ')

In [59]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [60]:
#import svm model and metrics
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [61]:
#SVM BASE LINE
model = SVC(kernel='linear', gamma='auto').fit(X_train, train_labels)
predictions = model.predict(X_test)

In [62]:
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      2436
           1       0.71      0.53      0.61      1064

    accuracy                           0.79      3500
   macro avg       0.76      0.72      0.73      3500
weighted avg       0.78      0.79      0.78      3500



In [63]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predictions)

0.6088362068965517

In [64]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predictions)

0.4813129456018161

In [65]:
# This is the name of the BERT model that we want to use.
model_name = 'bert-base-multilingual-cased'

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'

# This is the maximum number of tokens in any document sent to BERT.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'MasterMBERTMultilingual-saved'

In [66]:
from transformers import BertForSequenceClassification, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
#tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

In [67]:
train_labels


[1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [68]:
unique_labels = set(label for label in train_labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [69]:
label2id.keys()

dict_keys([0, 1])

In [70]:
id2label.keys()

dict_keys([0, 1])

In [71]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels]
test_labels_encoded  = [label2id[y] for y in test_labels]

In [72]:
set(train_labels_encoded)

{0, 1}

In [73]:
set(test_labels_encoded)

{0, 1}

In [74]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [75]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [76]:
# The model_name needs to match the name used for the tokenizer above.
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [77]:
#!pip install --upgrade accelerate

In [78]:
training_args = TrainingArguments(
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

In [79]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [80]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [81]:
torch.cuda.empty_cache()

In [82]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
100,0.640300,0.575456,0.719143
200,0.581500,0.556089,0.744857
300,0.569000,0.529448,0.731429
400,0.502300,0.522377,0.757429
500,0.546200,0.602894,0.731429
600,0.516700,0.494808,0.762571
700,0.527800,0.537180,0.765714
800,0.523000,0.473025,0.769714
900,0.524800,0.480153,0.778286
1000,0.484200,0.459002,0.772857


TrainOutput(global_step=3500, training_loss=0.43688821084158763, metrics={'train_runtime': 1508.2678, 'train_samples_per_second': 18.563, 'train_steps_per_second': 2.321, 'total_flos': 7366583327969280.0, 'train_loss': 0.43688821084158763, 'epoch': 2.0})

In [83]:
trainer.save_model(cached_model_directory_name)

In [84]:
trainer.evaluate()

{'eval_loss': 0.4758272171020508,
 'eval_accuracy': 0.8128571428571428,
 'eval_runtime': 14.5121,
 'eval_samples_per_second': 241.177,
 'eval_steps_per_second': 12.059,
 'epoch': 2.0}

In [85]:
predicted_results = trainer.predict(test_dataset)

In [86]:
predicted_results.predictions.shape

(3500, 2)

In [87]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [88]:
len(predicted_labels)

3500

Results for MBERT and Multilingual Dataset

In [89]:
print(classification_report(test_labels,
                            predicted_labels))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      2436
           1       0.72      0.63      0.67      1064

    accuracy                           0.81      3500
   macro avg       0.78      0.76      0.77      3500
weighted avg       0.81      0.81      0.81      3500



In [90]:
#calculate F1 score
from sklearn.metrics import f1_score
f1_score(test_labels, predicted_labels)

0.6706887883358471

In [91]:
from sklearn.metrics import matthews_corrcoef
#calculate Matthews correlation coefficient
matthews_corrcoef(test_labels, predicted_labels)

0.5434521717670842